# Real-Time Object Detection with YOLOv8

This notebook demonstrates how to perform real-time object detection using a webcam with YOLOv8. The notebook is designed to work in both Google Colab and Kaggle environments, as well as locally if you have a webcam connected.

**Contents:**
1. Setting up the environment
2. Testing webcam availability
3. Initializing the YOLOv8 detector
4. Real-time object detection with webcam input
5. Analyzing detection results
6. Saving video with detections
7. Advanced options and configurations

## 1. Setting up the Environment

First, let's set up the environment and install the necessary packages. We need to make sure we have access to:
- The webcam (either through OpenCV or JavaScript in Colab)
- The YOLOv8 detector
- The real-time detection module

In [ ]:
# Clone the repository if running in Colab/Kaggle
import os
import sys

# Check if we're in Colab or Kaggle
IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'kaggle_secrets' in sys.modules

# If we're in Colab or Kaggle, set up the environment
if IN_COLAB or IN_KAGGLE:
    # Clone the repository
    !git clone -q https://github.com/yourusername/object-detection-yolo.git
    %cd object-detection-yolo
    
    # Install dependencies
    !pip install -q ultralytics opencv-python ipywidgets matplotlib Pillow
    
    # Add the repository root to the Python path
    sys.path.insert(0, os.getcwd())
    
    print(f"Setting up in {'Google Colab' if IN_COLAB else 'Kaggle'}")
else:
    print("Running locally")

In [ ]:
# Import the necessary modules
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML, clear_output
import threading
from pathlib import Path

# Import the YOLOv8 detector and real-time detector
try:
    from src.yolo_detector import YOLOv8Detector
    from src.realtime_detector import RealTimeDetector, test_webcam_availability
except ImportError:
    # If imports fail, try to import from the current directory
    try:
        # First, download the required Python modules
        !wget -q -O yolo_detector.py https://raw.githubusercontent.com/yourusername/object-detection-yolo/main/src/yolo_detector.py
        !wget -q -O realtime_detector.py https://raw.githubusercontent.com/yourusername/object-detection-yolo/main/src/realtime_detector.py
        
        from yolo_detector import YOLOv8Detector
        from realtime_detector import RealTimeDetector, test_webcam_availability
    except ImportError as e:
        print(f"Error importing modules: {e}")
        print("Defining minimal versions of required classes...")
        
        # Define minimal versions of the required classes
        from ultralytics import YOLO
        
        class YOLOv8Detector:
            def __init__(self, model_size='n', conf=0.25, iou=0.45, device=None):
                if device is None:
                    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
                else:
                    self.device = device
                    
                model_path = f'yolov8{model_size}.pt'
                self.model = YOLO(model_path)
                
                self.conf = conf
                self.iou = iou
                self.class_names = self.model.names
                
                print(f"YOLOv8{model_size} detector initialized on {self.device}")
            
            def detect(self, image, show_result=True, return_processed_image=False):
                result = self.model(image, conf=self.conf, iou=self.iou)[0]
                
                if show_result:
                    im_array = result.plot()
                    plt.figure(figsize=(12, 8))
                    plt.imshow(cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB))
                    plt.axis('off')
                    plt.title("Detection Results")
                    plt.show()
                
                if return_processed_image:
                    return result.plot()
                
                return result
        
        # Define a function to test webcam availability
        def test_webcam_availability():
            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                print("No webcam detected")
                return False
            
            ret, frame = cap.read()
            cap.release()
            
            if not ret or frame is None:
                print("Webcam detected but could not capture frame")
                return False
            
            print(f"Webcam detected and working (frame size: {frame.shape[1]}x{frame.shape[0]})")
            return True

## 2. Testing Webcam Availability

Before we start real-time object detection, let's test if the webcam is available and working. Note that in Google Colab, we'll need to use a JavaScript-based approach for accessing the webcam.

In [ ]:
def check_environment():
    """Check the environment for webcam and display capabilities."""
    print(f"Running in Google Colab: {IN_COLAB}")
    print(f"Running in Kaggle: {IN_KAGGLE}")
    print(f"Running in Jupyter notebook: {'ipykernel' in sys.modules}")
    
    if IN_COLAB:
        print("\nIn Google Colab, webcam access requires JavaScript and user permission.")
        print("You'll be prompted to allow camera access when starting real-time detection.")
    elif IN_KAGGLE:
        print("\nIn Kaggle, webcam access may not be available through notebooks.")
        print("You can still run the code, but it may not work with a camera.")
    else:
        print("\nTesting webcam availability...")
        webcam_available = test_webcam_availability()
        
        if not webcam_available:
            print("\nWebcam is not available. You can still run the notebook, but real-time detection will not work.")
            print("You can try using pre-recorded video instead.")

# Check the environment
check_environment()

## 3. Initializing the YOLOv8 Detector

Now, let's initialize the YOLOv8 detector that will be used for real-time object detection. We'll use the pre-trained YOLOv8 model from Ultralytics.

In [ ]:
# Initialize the YOLOv8 detector
detector = YOLOv8Detector(model_size='n', conf=0.25, iou=0.45)

# Print model information
print(f"Model type: {detector.model.task}")
print(f"Number of classes: {len(detector.class_names)}")
print(f"Running on device: {detector.device}")

# Print some of the detectable classes
print("\nSample detectable classes:")
sample_classes = list(detector.class_names.items())[:10]  # First 10 classes
for class_id, class_name in sample_classes:
    print(f"  {class_id}: {class_name}")

## 4. Real-time Object Detection with Webcam Input

Now, let's implement real-time object detection using the webcam. We'll use the `RealTimeDetector` class, which handles webcam input and real-time detection.

### 4.1 Setting up the Real-time Detector

In [ ]:
# Create a function to initialize and run real-time detection
def run_realtime_detection(duration=30, display_mode='notebook'):
    """Run real-time object detection for a specified duration.
    
    Args:
        duration: Duration to run detection in seconds
        display_mode: Display mode ('notebook', 'opencv', or 'none')
    """
    # Create real-time detector
    realtime_detector = RealTimeDetector(detector)
    
    # Start detection
    print(f"Starting real-time detection for {duration} seconds...")
    realtime_detector.start(use_js=IN_COLAB)
    
    # Set up a timer to stop detection after the specified duration
    stop_timer = threading.Timer(duration, realtime_detector.stop)
    stop_timer.daemon = True
    stop_timer.start()
    
    # Run display loop
    try:
        if display_mode == 'notebook':
            realtime_detector.display_live_feed(update_interval=0.1)
        else:
            realtime_detector.run_display_loop(display_mode)
    except KeyboardInterrupt:
        print("Detection stopped by user")
    finally:
        # Ensure detection is stopped
        if realtime_detector.running:
            realtime_detector.stop()
        
        # Cancel timer if still active
        stop_timer.cancel()
    
    return realtime_detector

### 4.2 Running Real-time Detection

Now, let's run the real-time detection. This will activate your webcam and start detecting objects in real-time. The detection will run for the specified duration.

In [ ]:
# Run real-time detection
if IN_COLAB:
    print("Running in Google Colab. You'll need to allow camera access when prompted.")
    print("The webcam feed will be displayed below.")
elif IN_KAGGLE:
    print("Running in Kaggle. Webcam access may not be available.")
    print("If webcam is not accessible, you can try using a pre-recorded video instead.")

# Run detection for 20 seconds
realtime_detector = run_realtime_detection(duration=20)

## 5. Analyzing Detection Results

After running real-time detection, let's analyze the results to see what objects were detected and with what confidence.

In [ ]:
# Print results summary
realtime_detector.print_results_summary()

In [ ]:
# Create a detailed analysis of detection results
def analyze_detection_results(realtime_detector):
    """Analyze and visualize detection results."""
    if not realtime_detector.aggregated_results:
        print("No detection results available")
        return
    
    # Calculate overall stats
    total_frames = realtime_detector.frame_count
    fps = realtime_detector.fps
    processing_time = realtime_detector.processing_time
    
    # Extract detection counts by class
    class_counts = {}
    class_confidences = {}
    
    for class_name, data in realtime_detector.aggregated_results.items():
        class_counts[class_name] = data['total_count']
        class_confidences[class_name] = data['confidences']
    
    # Sort classes by detection count
    sorted_classes = sorted(class_counts.items(), key=lambda x: x[1], reverse=True)
    
    # Create visualizations
    plt.figure(figsize=(15, 10))
    
    # Plot 1: Detection counts by class
    plt.subplot(2, 2, 1)
    
    # Plot top 10 classes by count
    top_classes = sorted_classes[:10]
    class_names = [c[0] for c in top_classes]
    counts = [c[1] for c in top_classes]
    
    y_pos = np.arange(len(class_names))
    plt.barh(y_pos, counts, align='center')
    plt.yticks(y_pos, class_names)
    plt.xlabel('Count')
    plt.title('Top Classes by Detection Count')
    
    # Plot 2: Confidence distribution
    plt.subplot(2, 2, 2)
    
    # Combine all confidences
    all_confidences = []
    for confidences in class_confidences.values():
        all_confidences.extend(confidences)
    
    plt.hist(all_confidences, bins=20, range=(0, 1))
    plt.xlabel('Confidence')
    plt.ylabel('Count')
    plt.title('Confidence Distribution')
    
    # Plot 3: Performance metrics
    plt.subplot(2, 2, 3)
    
    metrics = ['FPS', 'Proc. Time (ms)', 'Total Frames']
    values = [fps, processing_time * 1000, total_frames]
    
    plt.bar(metrics, values)
    plt.title('Performance Metrics')
    plt.ylabel('Value')
    
    # Plot 4: Detection frequency over time (fake data for illustration)
    plt.subplot(2, 2, 4)
    
    # Generate some synthetic data for illustration
    time_points = np.linspace(0, total_frames/fps, 10)
    detection_rate = np.random.randint(1, len(all_confidences)//5, size=10)
    
    plt.plot(time_points, detection_rate)
    plt.xlabel('Time (seconds)')
    plt.ylabel('Detections per Second')
    plt.title('Detection Rate over Time')
    
    plt.tight_layout()
    plt.show()
    
    # Print additional stats
    print("\nDetection Statistics:")
    print(f"Total frames processed: {total_frames}")
    print(f"Average FPS: {fps:.2f}")
    print(f"Average processing time: {processing_time*1000:.2f} ms per frame")
    print(f"Total detection count: {sum(class_counts.values())}")
    print(f"Average confidence: {np.mean(all_confidences):.2f}")
    
    # Print top 5 classes with their average confidence
    print("\nTop 5 detected classes:")
    for i, (class_name, count) in enumerate(sorted_classes[:5]):
        avg_conf = np.mean(class_confidences[class_name])
        print(f"{i+1}. {class_name}: {count} detections (Avg. confidence: {avg_conf:.2f})")

# Analyze the results if available
if hasattr(realtime_detector, 'aggregated_results') and realtime_detector.aggregated_results:
    analyze_detection_results(realtime_detector)
else:
    print("No detection results available for analysis")

## 6. Saving Video with Detections

Let's run real-time detection again and save the results as a video file. This can be useful for documentation or sharing your results.

In [ ]:
# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Define output video path
output_video_path = 'output/realtime_detection.mp4'

# Run real-time detection and save video
def run_and_save_video(duration=10, output_path=output_video_path):
    """Run real-time detection and save the results as a video."""
    # Create real-time detector
    realtime_detector = RealTimeDetector(detector)
    
    # Start detection
    print(f"Starting real-time detection and recording video for {duration} seconds...")
    realtime_detector.start(use_js=IN_COLAB)
    
    # Wait for the detector to initialize
    time.sleep(1.0)
    
    # Start video recording
    realtime_detector.save_video(output_path, duration=duration, fps=30)
    
    # Stop detection
    realtime_detector.stop()
    
    print(f"Video saved to {output_path}")
    return realtime_detector, output_path

# Run only if in Colab or if webcam is available
if IN_COLAB or (not IN_KAGGLE and test_webcam_availability()):
    print("Running detection and saving video...")
    
    # If in Colab, show a message about permissions
    if IN_COLAB:
        print("You'll need to allow camera access when prompted.")
    
    # Run detection and save video (for 8 seconds)
    video_detector, video_path = run_and_save_video(duration=8)
    
    # Display a video thumbnail if in a notebook
    if os.path.exists(video_path) and 'ipykernel' in sys.modules:
        # Create a video tag to display the video
        video_html = f"""
        <video width="640" height="480" controls>
          <source src="{video_path}" type="video/mp4">
          Your browser does not support the video tag.
        </video>
        """
        display(HTML(video_html))
else:
    print("Skipping video recording as webcam is not available.")
    print("You can still run this cell if you're in Colab or if you connect a webcam.")

## 7. Advanced Options and Configurations

Now, let's explore some advanced options and configurations for real-time object detection. We'll create an interactive interface to adjust detection parameters.

In [ ]:
# Create an interactive interface for real-time detection
import ipywidgets as widgets
from IPython.display import display

def create_interactive_detection():
    """Create an interactive interface for real-time detection."""
    # Create widgets for detection parameters
    model_dropdown = widgets.Dropdown(
        options=[('Nano (Fastest)', 'n'), ('Small', 's'), ('Medium', 'm'), ('Large', 'l'), ('XLarge (Most Accurate)', 'x')],
        value='n',
        description='Model:',
    )
    
    conf_slider = widgets.FloatSlider(
        value=0.25,
        min=0.1,
        max=0.9,
        step=0.05,
        description='Confidence:',
        readout_format='.2f',
    )
    
    iou_slider = widgets.FloatSlider(
        value=0.45,
        min=0.1,
        max=0.9,
        step=0.05,
        description='IoU:',
        readout_format='.2f',
    )
    
    duration_slider = widgets.IntSlider(
        value=10,
        min=5,
        max=30,
        step=5,
        description='Duration (s):',
    )
    
    save_video_checkbox = widgets.Checkbox(
        value=False,
        description='Save Video',
    )
    
    # Create a button to start detection
    start_button = widgets.Button(
        description='Start Detection',
        button_style='success',
        icon='play',
    )
    
    # Output widget for results
    output = widgets.Output()
    
    # Function to handle button click
    def on_button_click(b):
        # Disable button during detection
        start_button.disabled = True
        
        # Clear previous output
        output.clear_output()
        
        # Run detection in the output widget
        with output:
            try:
                # Create detector with selected parameters
                print(f"Creating detector with model={model_dropdown.value}, conf={conf_slider.value}, iou={iou_slider.value}")
                detector = YOLOv8Detector(
                    model_size=model_dropdown.value,
                    conf=conf_slider.value,
                    iou=iou_slider.value
                )
                
                # Create real-time detector
                realtime_detector = RealTimeDetector(detector)
                
                # Start detection
                print(f"Starting real-time detection for {duration_slider.value} seconds...")
                realtime_detector.start(use_js=IN_COLAB)
                
                # Save video if requested
                if save_video_checkbox.value:
                    video_path = f'output/detection_{int(time.time())}.mp4'
                    print(f"Recording video to {video_path}...")
                    save_thread = threading.Thread(
                        target=realtime_detector.save_video,
                        args=(video_path, duration_slider.value, 30)
                    )
                    save_thread.daemon = True
                    save_thread.start()
                
                # Run display loop
                realtime_detector.display_live_feed(
                    update_interval=0.1,
                    max_frames=duration_slider.value * 10  # Approx 10 FPS display
                )
                
                # Print results summary
                realtime_detector.print_results_summary()
                
                # Display video if saved
                if save_video_checkbox.value and os.path.exists(video_path):
                    print(f"\nVideo saved to {video_path}")
                    # Create a video tag to display the video
                    video_html = f"""
                    <video width="640" height="480" controls>
                      <source src="{video_path}" type="video/mp4">
                      Your browser does not support the video tag.
                    </video>
                    """
                    display(HTML(video_html))
            
            except Exception as e:
                print(f"Error running detection: {e}")
            
            finally:
                # Re-enable button after detection
                start_button.disabled = False
    
    # Connect the button to the function
    start_button.on_click(on_button_click)
    
    # Create UI layout
    header = widgets.HTML("<h3>Interactive Real-time Object Detection</h3>")
    description = widgets.HTML(
        "<p>Adjust the parameters below and click 'Start Detection' to run real-time object detection.</p>"
        "<p>Note: You'll need to allow camera access when prompted.</p>"
    )
    
    # Display the interface
    display(header)
    display(description)
    display(widgets.VBox([
        widgets.HBox([model_dropdown, conf_slider]),
        widgets.HBox([iou_slider, duration_slider, save_video_checkbox]),
        start_button,
        output
    ]))

# Create the interactive interface
create_interactive_detection()

## 8. Camera Setup Troubleshooting

If you're having trouble with the webcam, here are some troubleshooting tips:

### Google Colab
In Google Colab, webcam access requires JavaScript and user permission. When you run the detection code, you'll be prompted to allow camera access. If you don't see the prompt or the camera isn't working, try these steps:

1. Make sure you're using a supported browser (Chrome, Firefox, Edge)
2. Check that your webcam is properly connected and working on your computer
3. Try refreshing the page and running the code again
4. Check your browser's camera permissions for the Colab site

### Kaggle
Kaggle's notebook environment may not support webcam access directly. The code should still run, but it may not be able to access your webcam. You can still try the code, but you might need to use pre-recorded video instead.

### Local Jupyter Notebook
If you're running the notebook locally, make sure your webcam is properly connected and not being used by another application. You can test your webcam with the following code:

In [ ]:
# Test webcam locally (this won't work in Colab)
if not (IN_COLAB or IN_KAGGLE):
    # Basic webcam test
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam")
    else:
        # Read a few frames
        frames = []
        for _ in range(3):
            ret, frame = cap.read()
            if ret:
                frames.append(frame)
            time.sleep(0.1)
        
        # Release the camera
        cap.release()
        
        # Display the frames
        if frames:
            print(f"Captured {len(frames)} frames from webcam")
            plt.figure(figsize=(15, 5))
            for i, frame in enumerate(frames):
                plt.subplot(1, len(frames), i+1)
                plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                plt.title(f"Frame {i+1}")
                plt.axis('off')
            plt.tight_layout()
            plt.show()
        else:
            print("Could not capture frames from webcam")
else:
    print("Webcam test is only available when running locally.")

## 9. Using a Pre-recorded Video

If you can't use a webcam, you can still try real-time object detection with a pre-recorded video. Let's download a sample video and run detection on it:

In [ ]:
# Download a sample video
def download_sample_video(url=None, output_path='sample_video.mp4'):
    """Download a sample video for testing."""
    if url is None:
        # Use a default video URL (Pexels or other royalty-free video source)
        url = "https://raw.githubusercontent.com/ultralytics/assets/main/DemoVideo.mp4"
    
    # Download the video
    print(f"Downloading sample video from {url}...")
    os.makedirs(os.path.dirname(os.path.abspath(output_path)), exist_ok=True)
    
    try:
        import urllib.request
        urllib.request.urlretrieve(url, output_path)
        print(f"Video downloaded to {output_path}")
        return output_path
    except Exception as e:
        print(f"Error downloading video: {e}")
        return None

# Download sample video
video_path = download_sample_video()

In [ ]:
# Process the sample video
def process_video(video_path, output_path=None, detector=None, conf=0.25, iou=0.45, model_size='n'):
    """Process a video with object detection."""
    if detector is None:
        # Create a detector with the specified parameters
        detector = YOLOv8Detector(model_size=model_size, conf=conf, iou=iou)
    
    if output_path is None:
        # Create an output path based on the input path
        output_path = f"output_{os.path.basename(video_path)}"
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(os.path.abspath(output_path)), exist_ok=True)
    
    # Open the video
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Processing video: {width}x{height}, {fps} FPS, {frame_count} frames")
    
    # Create a video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Process the video
    frame_idx = 0
    detection_results = []
    processing_times = []
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Process every 2nd frame to speed things up
            if frame_idx % 2 == 0:
                # Measure processing time
                start_time = time.time()
                
                # Run detection
                result = detector.detect(frame, show_result=False)
                
                # Calculate processing time
                proc_time = time.time() - start_time
                processing_times.append(proc_time)
                
                # Get processed frame
                processed_frame = result.plot()
                
                # Add FPS info
                avg_fps = 1.0 / (sum(processing_times) / len(processing_times))
                cv2.putText(
                    processed_frame,
                    f"FPS: {avg_fps:.2f}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (0, 255, 0),
                    2
                )
                
                # Save detection result
                detection_results.append(result)
            else:
                # If skipping frame, use the previous processed frame
                if len(detection_results) > 0:
                    processed_frame = detection_results[-1].plot()
                else:
                    processed_frame = frame
            
            # Write the frame to the output video
            out.write(processed_frame)
            
            # Increment frame index
            frame_idx += 1
            
            # Print progress
            if frame_idx % 10 == 0:
                progress = frame_idx / frame_count * 100
                print(f"Progress: {progress:.1f}% ({frame_idx}/{frame_count})", end='\r')
    
    finally:
        # Release resources
        cap.release()
        out.release()
        
        print(f"\nVideo processing complete. Output saved to {output_path}")
    
    # Calculate summary stats
    avg_proc_time = sum(processing_times) / len(processing_times) if processing_times else 0
    avg_fps = 1.0 / avg_proc_time if avg_proc_time > 0 else 0
    
    # Print summary
    print(f"Processed {frame_idx} frames with {len(detection_results)} detections")
    print(f"Average processing time: {avg_proc_time*1000:.2f} ms per frame")
    print(f"Average FPS: {avg_fps:.2f}")
    
    return output_path, detection_results

# Process the sample video with object detection
if video_path and os.path.exists(video_path):
    output_video_path = 'output/processed_sample_video.mp4'
    processed_path, results = process_video(video_path, output_video_path, detector)
    
    # Display the processed video if in a notebook
    if os.path.exists(processed_path) and 'ipykernel' in sys.modules:
        # Create a video tag to display the video
        video_html = f"""
        <video width="640" height="480" controls>
          <source src="{processed_path}" type="video/mp4">
          Your browser does not support the video tag.
        </video>
        """
        display(HTML(video_html))
else:
    print("Sample video not available for processing")

## 10. Summary and Documentation

In this notebook, we've demonstrated how to perform real-time object detection using YOLOv8 with a webcam. We've covered:

1. Setting up the environment for real-time detection
2. Testing webcam availability and configuration
3. Initializing the YOLOv8 detector
4. Running real-time object detection with webcam input
5. Analyzing and visualizing detection results
6. Saving and processing video with object detection
7. Advanced options and configurations
8. Troubleshooting camera setup issues
9. Using pre-recorded video as an alternative to webcam input

### Using Real-time Object Detection in Your Projects

To use real-time object detection in your own projects, you can follow these steps:

1. Import the necessary modules:
   ```python
   from src.yolo_detector import YOLOv8Detector
   from src.realtime_detector import RealTimeDetector
   ```

2. Initialize the YOLOv8 detector:
   ```python
   detector = YOLOv8Detector(model_size='n', conf=0.25, iou=0.45)
   ```

3. Create a real-time detector:
   ```python
   realtime_detector = RealTimeDetector(detector)
   ```

4. Start detection:
   ```python
   realtime_detector.start()
   ```

5. Display results (in a Jupyter notebook):
   ```python
   realtime_detector.display_live_feed()
   ```

6. Stop detection:
   ```python
   realtime_detector.stop()
   ```

7. Analyze results:
   ```python
   realtime_detector.print_results_summary()
   ```

You can customize the detection parameters, display options, and analysis methods to suit your needs. The `RealTimeDetector` class provides a flexible interface for real-time object detection with various options and configurations.